<a href="https://colab.research.google.com/github/krishnamohanathota/GenerativeAI/blob/main/langchain/modules/L2_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Memory Module

https://python.langchain.com/docs/modules/memory/

By default, Chains and Agents are stateless, meaning that they treat each incoming query independently (like the underlying LLMs and chat models themselves). In some applications, like chatbots, it is essential to remember previous interactions, both in the short and long-term. The **Memory** class does exactly that.

LangChain provides memory components in two forms:
- Provides helper utilities for managing and manipulating previous chat messages
- Provides easy ways to incorporate these utilities into chains

Few memory types:
- ChatMessageHistory
- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryBufferMemory

LangChain actually supports additional memory types as well.
One of the most powerful is **vector** data memory,
if you're familiar with word embeddings and text embeddings, the
vector database actually stores such embeddings.


In [2]:
!pip install python-dotenv
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00


In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # Read local .env files
openai.api_key = os.environ["OPENAI_API_KEY"]

## ChatMessageHistory

Lightweight wrapper which exposes convenience methods for saving **Human messages**, **AI messages**, and then fetching them all.

In [9]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory();

history.add_user_message("Hi..")
history.add_ai_message("Whats up ?")

print(history)
print(history.messages)

messages=[HumanMessage(content='Hi..', additional_kwargs={}, example=False), AIMessage(content='Whats up ?', additional_kwargs={}, example=False)]
[HumanMessage(content='Hi..', additional_kwargs={}, example=False), AIMessage(content='Whats up ?', additional_kwargs={}, example=False)]


## ConversationBufferMemory

wrapper around **ChatMessageHistory** that extracts the messages in a variable.

In [17]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
print(memory)


memory.chat_memory.add_user_message("Hi!")
memory.chat_memory.add_ai_message("Whats up ???")

print(memory)

## {'history': 'Human: Hi!\nAI: Whats up ???'}
print(memory.load_memory_variables({}))

print("------")

## Get the history as a list of messages
memory = ConversationBufferMemory(return_messages=True)

print(memory)

memory.chat_memory.add_user_message("Hi!")
memory.chat_memory.add_ai_message("Whats up ???")

print(memory)

## {'history': [HumanMessage(content='Hi!', additional_kwargs={}, example=False), AIMessage(content='Whats up ???', additional_kwargs={}, example=False)]}
print(memory.load_memory_variables({}))

chat_memory=ChatMessageHistory(messages=[]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history'
chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi!', additional_kwargs={}, example=False), AIMessage(content='Whats up ???', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history'
{'history': 'Human: Hi!\nAI: Whats up ???'}
------
chat_memory=ChatMessageHistory(messages=[]) output_key=None input_key=None return_messages=True human_prefix='Human' ai_prefix='AI' memory_key='history'
chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi!', additional_kwargs={}, example=False), AIMessage(content='Whats up ???', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=True human_prefix='Human' ai_prefix='AI' memory_key='history'
{'history': [HumanMessage(content='Hi!', additional_kwargs={}, 

### Using in a Chain

In [57]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(temperature=0.0)
memory = ConversationBufferMemory();
print(llm)

# setting "verbose=True" so we can see the prompt
conversation = ConversationChain(llm=llm, verbose=True, memory = memory)

## Don't print "conversation" as this is having OpenAI API KEY

conversation.predict(input="Hi there")


OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.0, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there
AI:

> Finished chain.


' Hi there! How can I help you?'

In [58]:
conversation.predict(input="I'm doing well! Which country won more FIFA worldcups ? ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi there! How can I help you?
Human: I'm doing well! Which country won more FIFA worldcups ? 
AI:

> Finished chain.


' Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.'

In [59]:
conversation.predict(input="Who is the most popular player?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi there! How can I help you?
Human: I'm doing well! Which country won more FIFA worldcups ? 
AI:  Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.
Human: Who is the most popular player?
AI:

> Finished chain.


" The most popular player in the world is Lionel Messi. He has won the Ballon d'Or six times, and is widely considered to be one of the greatest players of all time."

In [60]:
conversation.predict(input="Who had won most awards?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi there! How can I help you?
Human: I'm doing well! Which country won more FIFA worldcups ? 
AI:  Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.
Human: Who is the most popular player?
AI:  The most popular player in the world is Lionel Messi. He has won the Ballon d'Or six times, and is widely considered to be one of the greatest players of all time.
Human: Who had won most awards?
AI:



> Finished chain.


" The player who has won the most awards is Cristiano Ronaldo. He has won five Ballon d'Or awards, four European Golden Shoes, and numerous other awards."

In [61]:
print(type(conversation.memory.chat_memory))
print(type(memory.chat_memory))

print(conversation.memory.chat_memory)
print(memory.chat_memory)

print(conversation.memory.chat_memory.messages)
print(type(conversation.memory.chat_memory.messages))

<class 'langchain.memory.chat_message_histories.in_memory.ChatMessageHistory'>
<class 'langchain.memory.chat_message_histories.in_memory.ChatMessageHistory'>
messages=[HumanMessage(content='Hi there', additional_kwargs={}, example=False), AIMessage(content=' Hi there! How can I help you?', additional_kwargs={}, example=False), HumanMessage(content="I'm doing well! Which country won more FIFA worldcups ? ", additional_kwargs={}, example=False), AIMessage(content=' Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.', additional_kwargs={}, example=False), HumanMessage(content='Who is the most popular player?', additional_kwargs={}, example=False), AIMessage(content=" The most popular player in the world is Lionel Messi. He has won the Ballon d'Or six times, and is widely considered to be one of the greatest players of all time.", additional_kwargs={}, example=False), HumanMessage(content='Who had won most awards?', additiona

### Saving Message History

You may often have to save messages, and then load them to use again. This can be done easily by first converting the messages to normal python dictionaries, saving those (as json or something) and then loading those

In [62]:
from langchain.schema import messages_from_dict, messages_to_dict

dict = messages_to_dict(conversation.memory.chat_memory.messages)

print(dict)
print(type(dict))

[{'type': 'human', 'data': {'content': 'Hi there', 'additional_kwargs': {}, 'example': False}}, {'type': 'ai', 'data': {'content': ' Hi there! How can I help you?', 'additional_kwargs': {}, 'example': False}}, {'type': 'human', 'data': {'content': "I'm doing well! Which country won more FIFA worldcups ? ", 'additional_kwargs': {}, 'example': False}}, {'type': 'ai', 'data': {'content': ' Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.', 'additional_kwargs': {}, 'example': False}}, {'type': 'human', 'data': {'content': 'Who is the most popular player?', 'additional_kwargs': {}, 'example': False}}, {'type': 'ai', 'data': {'content': " The most popular player in the world is Lionel Messi. He has won the Ballon d'Or six times, and is widely considered to be one of the greatest players of all time.", 'additional_kwargs': {}, 'example': False}}, {'type': 'human', 'data': {'content': 'Who had won most awards?', 'additional_kwa

In [63]:
new_messages = messages_from_dict(dict)

print(new_messages)
print(type(new_messages))

[HumanMessage(content='Hi there', additional_kwargs={}, example=False), AIMessage(content=' Hi there! How can I help you?', additional_kwargs={}, example=False), HumanMessage(content="I'm doing well! Which country won more FIFA worldcups ? ", additional_kwargs={}, example=False), AIMessage(content=' Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.', additional_kwargs={}, example=False), HumanMessage(content='Who is the most popular player?', additional_kwargs={}, example=False), AIMessage(content=" The most popular player in the world is Lionel Messi. He has won the Ballon d'Or six times, and is widely considered to be one of the greatest players of all time.", additional_kwargs={}, example=False), HumanMessage(content='Who had won most awards?', additional_kwargs={}, example=False), AIMessage(content=" The player who has won the most awards is Cristiano Ronaldo. He has won five Ballon d'Or awards, four European Golden 

In [64]:
print(memory.buffer)

Human: Hi there
AI:  Hi there! How can I help you?
Human: I'm doing well! Which country won more FIFA worldcups ? 
AI:  Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.
Human: Who is the most popular player?
AI:  The most popular player in the world is Lionel Messi. He has won the Ballon d'Or six times, and is widely considered to be one of the greatest players of all time.
Human: Who had won most awards?
AI:  The player who has won the most awards is Cristiano Ronaldo. He has won five Ballon d'Or awards, four European Golden Shoes, and numerous other awards.


In [65]:
# Return history buffer. {} --> This means empty dicto
memory.load_memory_variables({})

{'history': "Human: Hi there\nAI:  Hi there! How can I help you?\nHuman: I'm doing well! Which country won more FIFA worldcups ? \nAI:  Brazil has won the most FIFA World Cups, with a total of five titles. They won in 1958, 1962, 1970, 1994, and 2002.\nHuman: Who is the most popular player?\nAI:  The most popular player in the world is Lionel Messi. He has won the Ballon d'Or six times, and is widely considered to be one of the greatest players of all time.\nHuman: Who had won most awards?\nAI:  The player who has won the most awards is Cristiano Ronaldo. He has won five Ballon d'Or awards, four European Golden Shoes, and numerous other awards."}

In [69]:
# If you would like to add a data manually to memory
memory1 = ConversationBufferMemory();
memory1.save_context({"input" : "Hi"},
          {"output":"What's up?"})
memory1.buffer

"Human: Hi\nAI: What's up?"

In [70]:
memory1.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up?"}

In [73]:
memory1.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory1.buffer

"Human: Hi\nAI: What's up?\nHuman: Not much, just hanging\nAI: Cool"

In [74]:
memory1.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up?\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

It is a type of memory that stores a fixed-size window of the conversation history. It keeps track of the most recent messages in the conversation and discards older messages as new ones are added. This memory type is useful when you only need to remember a limited amount of context from the conversation.

In [76]:
from langchain.memory import ConversationBufferWindowMemory

In [77]:
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input" : "Hi"},
          {"output":"What's up?"})
memory.buffer

[HumanMessage(content='Hi', additional_kwargs={}, example=False),
 AIMessage(content="What's up?", additional_kwargs={}, example=False)]

In [78]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.buffer

[HumanMessage(content='Hi', additional_kwargs={}, example=False),
 AIMessage(content="What's up?", additional_kwargs={}, example=False),
 HumanMessage(content='Not much, just hanging', additional_kwargs={}, example=False),
 AIMessage(content='Cool', additional_kwargs={}, example=False)]

In [79]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [81]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferWindowMemory

llm = OpenAI(temperature=0.0)

#If I set memory to conversational buffer window memory with k equals one,
#the variable k equals one specifies that I wanted to remember just
#one conversational exchange.
memory = ConversationBufferWindowMemory(k=1)

# setting "verbose=True" so we can see the prompt
conversation = ConversationChain(llm=llm, verbose=True, memory = memory)

conversation.predict(input = "Hi, My name is Krishna Mohan")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, My name is Krishna Mohan
AI:

> Finished chain.


" Hi Krishna Mohan, my name is AI. It's nice to meet you. What can I do for you today?"

In [82]:
conversation.predict(input = "What is my name ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, My name is Krishna Mohan
AI:  Hi Krishna Mohan, my name is AI. It's nice to meet you. What can I do for you today?
Human: What is my name ?
AI:

> Finished chain.


' Your name is Krishna Mohan. Is there anything else I can help you with?'

In [83]:
conversation.predict(input = "What is 1 + 1 ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is my name ?
AI:  Your name is Krishna Mohan. Is there anything else I can help you with?
Human: What is 1 + 1 ?
AI:

> Finished chain.


' 1 + 1 is equal to 2. Is there anything else I can help you with?'

In [84]:
# Here it won't be able to return the name as window size was set to "1" for ConversationBufferWindowMemory
# Because k equals 1, it only remembers the  last exchange versus what is 1 + 1?
# The answer is 1 + 1 equals 2, and it's forgotten this early exchange which is now, now says, "I'm sorry, I don't know your name."
conversation.predict(input = "What is my name ?")

# Observe the below outout, old memory has dropped



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1 + 1 ?
AI:  1 + 1 is equal to 2. Is there anything else I can help you with?
Human: What is my name ?
AI:

> Finished chain.


" I'm sorry, I don't know your name."

In [86]:
type(memory)

langchain.memory.buffer_window.ConversationBufferWindowMemory

In [87]:
#But memory buffer has all the history. Noy sure why ????
memory.buffer

[HumanMessage(content='Hi, My name is Krishna Mohan', additional_kwargs={}, example=False),
 AIMessage(content=" Hi Krishna Mohan, my name is AI. It's nice to meet you. What can I do for you today?", additional_kwargs={}, example=False),
 HumanMessage(content='What is my name ?', additional_kwargs={}, example=False),
 AIMessage(content=' Your name is Krishna Mohan. Is there anything else I can help you with?', additional_kwargs={}, example=False),
 HumanMessage(content='What is 1 + 1 ?', additional_kwargs={}, example=False),
 AIMessage(content=' 1 + 1 is equal to 2. Is there anything else I can help you with?', additional_kwargs={}, example=False),
 HumanMessage(content='What is my name ?', additional_kwargs={}, example=False),
 AIMessage(content=" I'm sorry, I don't know your name.", additional_kwargs={}, example=False)]

## ConversationTokenBufferMemory

https://python.langchain.com/docs/modules/memory/how_to/token_buffer

It keeps a buffer of recent interactions in memory, and uses **token length** rather than number of interactions to determine when to flush interactions.

It is a type of memory that can be used to store conversation tokens in a conversation chain. It allows the conversation to be **resumed** from a specific point by providing the conversation tokens as input.

It will store the conversation tokens generated during the conversation and allow you to continue the conversation from where you left off. This can be useful when you want to **resume** a conversation after a break or when you want to have a back-and-forth conversation with the AI.

In [4]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00


In [10]:
from langchain.llms import OpenAI
from langchain.memory import ConversationTokenBufferMemory

llm = OpenAI(temperature=0.0)
# To get the history as a list of messages
memory = ConversationTokenBufferMemory(llm = llm, max_token_limit=30, return_messages=True)
#memory = ConversationTokenBufferMemory(llm = llm, max_token_limit=30)

memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

memory.load_memory_variables({})


{'history': [AIMessage(content='Amazing!', additional_kwargs={}, example=False),
  HumanMessage(content='Backpropagation is what?', additional_kwargs={}, example=False),
  AIMessage(content='Beautiful!', additional_kwargs={}, example=False),
  HumanMessage(content='Chatbots are what?', additional_kwargs={}, example=False),
  AIMessage(content='Charming!', additional_kwargs={}, example=False)]}

In [13]:
from langchain.chains import ConversationChain
conversation_with_summary = ConversationChain(llm = llm, memory=memory, verbose=True)

conversation_with_summary.predict(input="Hi, how are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[AIMessage(content='Amazing!', additional_kwargs={}, example=False), HumanMessage(content='Backpropagation is what?', additional_kwargs={}, example=False), AIMessage(content='Beautiful!', additional_kwargs={}, example=False), HumanMessage(content='Chatbots are what?', additional_kwargs={}, example=False), AIMessage(content='Charming!', additional_kwargs={}, example=False)]
Human: Hi, how are you?
AI:

> Finished chain.


" Hi there! I'm doing great, thanks for asking. How about you?"

In [14]:
conversation_with_summary.predict(input = "Who is the founder of OpenAI ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, how are you?', additional_kwargs={}, example=False), AIMessage(content=" Hi there! I'm doing great, thanks for asking. How about you?", additional_kwargs={}, example=False)]
Human: Who is the founder of OpenAI ?
AI:

> Finished chain.


' The founder of OpenAI is Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, and Wojciech Zaremba.'

In [15]:
conversation_with_summary.predict(input="What is the use with LangChain framework?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: What is the use with LangChain framework?
AI:

> Finished chain.


' The LangChain framework is a blockchain-based platform that enables developers to create decentralized applications (dApps) that are powered by natural language processing (NLP). It provides a secure, reliable, and scalable infrastructure for developers to build applications that can interact with users in natural language. It also provides a set of tools and services to help developers create and deploy their applications quickly and easily.'

In [ ]:
# Please observe above response. some histiory (tokens) are removed

## ConversationSummaryBufferMemory

https://python.langchain.com/docs/modules/memory/how_to/summary_buffer

- It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions, it compiles them into a summary
- It uses token length rather than number of interactions to determine when to flush interactions.

In [17]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

In [19]:
llm = OpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm = llm, max_token_limit=30, return_messages=True)

memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human greets the AI and asks what is on the schedule for the day. The AI responds that there is a meeting with the product team at 8am, followed by time to work on the LangChain project, and then lunch with a customer at a nearby Italian restaurant to discuss the latest in AI. The human should bring their laptop to show the latest LLM demo.', additional_kwargs={})]}

In [ ]:
#memory
#Please don't print this as it is printing OpenAI key also

In [22]:
#Using predict_new_summary diretcly
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.'

In [24]:
from langchain.chains import ConversationChain

llm = OpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm = llm, max_token_limit=30, return_messages=True)
conversation_with_summary = ConversationChain(llm = llm, memory = memory, verbose=True)

conversation_with_summary.predict(input = "Hi, How are you? ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, How are you? 
AI:

> Finished chain.


" Hi there! I'm doing great, thanks for asking. How about you?"

In [25]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, How are you? ', additional_kwargs={}, example=False), AIMessage(content=" Hi there! I'm doing great, thanks for asking. How about you?", additional_kwargs={}, example=False)]
Human: Just working on writing some documentation!
AI:

> Finished chain.


' That sounds like a great use of your time! What kind of documentation are you writing?'

In [26]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content="\nThe human asks the AI how it is doing, to which the AI responds that it is doing great and inquires about the human's wellbeing.", additional_kwargs={}), HumanMessage(content='Just working on writing some documentation!', additional_kwargs={}, example=False), AIMessage(content=' That sounds like a great use of your time! What kind of documentation are you writing?', additional_kwargs={}, example=False)]
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


" No, I haven't heard of LangChain. Could you tell me more about it?"

In [27]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="LangChain is a `framework` for developing applications powered by language models.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content="\nThe human asks the AI how it is doing, to which the AI responds that it is doing great and inquires about the human's wellbeing. The human responds that they are working on writing documentation for LangChain, to which the AI responds positively and inquires about the kind of documentation being written.", additional_kwargs={}), AIMessage(content=" No, I haven't heard of LangChain. Could you tell me more about it?", additional_kwargs={}, example=False)]
Human: LangChain is a `framework` for developing applications powered by language models.
AI:

> Finished chain.


' Interesting! What kind of applications can be developed with LangChain?'